In [1]:
import xarray as xr
import glob
import numpy as np
import pandas as pd

In [2]:
import sys
sys.path.append('/glade/u/home/linnia/ctsm6_ppe/')
from utils.pyfunctions import *

utils_path = '/glade/u/home/linnia/ctsm6_ppe/utils/'

In [3]:
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client
ncores=1
nmem='10GB'
cluster = PBSCluster(
    cores=ncores, # The number of cores you want
    memory=nmem, # Amount of memory
    processes=1, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='$TMPDIR', # Use your local directory
    resource_spec='select=1:ncpus='+str(ncores)+':mem='+nmem, # Specify resources
    project='P93300641', # Input your project ID here
    walltime='01:00:00', # Amount of wall time
    #interface='ib0', # Interface to use
)

# Scale up
cluster.scale(10)

# Setup your client
client = Client(cluster)

/glade/u/home/linnia/miniconda3/envs/ppe-py/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [4]:
yy = (1985,2023)
dir='/glade/campaign/cgd/tss/projects/PPE/ctsm6_oaat/hist/'
key = '/glade/campaign/cgd/tss/projects/PPE/ctsm6_oaat/ctsm6_oaat_key.csv'
exp='oaat'
tape='h0'

dvs=['GPP','AR','HR','NPP','NBP','NEP','ER',
     'EFLX_LH_TOT','FCTR','FCEV','FGEV','BTRANMN','FGR','FSH',
     'SOILWATER_10CM','TWS','QRUNOFF','SNOWDP','H2OSNO','FSNO',
     'TLAI','FSR','ALTMAX','TV','TG','NPP_NUPTAKE'
     'FAREA_BURNED','COL_FIRE_CLOSS',
     'TOTVEGC','TOTECOSYSC','TOTSOMC_1m',
     'TOTVEGN','TOTECOSYSN']

In [5]:
ds= get_exp(exp,dir,key,dvs,'h0',yy,utils_path)

In [6]:
# calculate global and biome mean, std, amp
la=xr.open_dataset(utils_path+'landarea_retrain_h0.nc').landarea
b=xr.open_dataset(utils_path+'whit/whitkey_CRUJRA.nc').biome
out=xr.Dataset()
    
for v in dvs:

        x=amean(ds[v])
        amp=(ds[v].groupby('time.year').max()-ds[v].groupby('time.year').min()).mean(dim='year').compute()

        out[v+'_gridded_mean']=x.mean(dim='year')

        out[v+'_global_amp'] =gmean(amp,la)
        out[v+'_global_std'] =gmean(x.std(dim='year'),la)
        out[v+'_global_mean']=gmean(x.mean(dim='year'),la)

        out[v+'_biome_amp']  =bmean(amp,la,b)
        out[v+'_biome_std']  =bmean(x.std(dim='year'),la,b)
        out[v+'_biome_mean'] =bmean(x.mean(dim='year'),la,b)

        for dv in out.data_vars:
            if v in dv:
                out[dv].attrs=ds[v].attrs

In [7]:
df_key = pd.read_csv(key)

out = out.assign_coords(
    key=('ens', df_key['key'].values),
    param=('ens', df_key['param'].values),
    minmax=('ens', df_key['minmax'].values)
)

out = out.set_index(ens=["param", "minmax", "key"], append=True)
ds_out = out.reset_index("ens")

In [12]:
ds_out.to_netcdf('/glade/campaign/cgd/tss/projects/PPE/ctsm6_oaat/postp/ctsm6_oaat_postp_1985-2023.nc')